In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
data_folder = './data/'
sea_state = 3 # Can be 1, 2 or 3
realization = 7 # Can be 1, 2, ... 8

prediction_time=5 #in s.

filename = 'Ship_{:d}_{:d}.csv'.format(sea_state,realization)
input_data = pd.read_csv(data_folder+filename)
import pandas as pd
df=pd.DataFrame(input_data)

# plt.plot(df['Rot_Z'][0:5000])
# plt.plot(df['Z'])

In [2]:
# FFT of the rotation pattern.

from scipy.fft import fft, ifft
import numpy as np

# Number of sample points

time=df['Time']

X=df['X'].to_numpy()
Y=df['Y'].to_numpy()
Z=df['Z'].to_numpy()


Rot_X= df['Rot_X'].values.tolist()
Rot_Y= df['Rot_Y'].values.tolist()
Rot_Z= df['Rot_Z'].values.tolist()

time_train=time[0:19980]
time_test=time[19980:24000]


X_train=X[0:19980]
X_test=X[19980:-1]

Y_train=Y[0:19980]
Y_test=Y[19980:-1]

Z_train=Z[0:19980]
Z_test=Z[19980:-1]

Rot_X_train=Rot_X[0:19980]
Rot_X_test=Rot_X[19980:24000]

Rot_Y_train=Rot_Y[0:19980]
Rot_Y_test=Rot_Y[19980:24000]

Rot_Z_train=Rot_Z[0:19980]
Rot_Z_test=Rot_Z[19980:24000]

max_X_train=max(X_train)
max_Y_train=max(Y_train)
max_Z_train=max(Z_train)



max_Rot_X_train=max(Rot_X_train)
max_Rot_Y_train=max(Rot_Y_train)
max_Rot_Z_train=max(Rot_Z_train)

# Rot_Y=df['Rot_Y'].to_numpy()

xf = np.linspace(0.0, 19980/20, 19980//60)

fft_Rot_X_train = fft(Rot_X_train)
fft_Rot_Y_train = fft(Rot_Y_train)
fft_Rot_Z_train = fft(Rot_Z_train)

# fft_Rot_Y_short = np.linspace(0.0, 20, 400)
# fft_Rot_X_short = np.linspace(0.0, 20, 400)
fft_Rot_X_short=(2/19980*fft_Rot_X_train[0:400])
fft_Rot_Y_short=(2/19980*fft_Rot_Y_train[0:400])
fft_Rot_Z_short=(2/19980*fft_Rot_Z_train[0:400])

# max_fft_Rot_Y_short=max(fft_Rot_Y_short)
# max_fft_Rot_Y_short=max(fft_Rot_Y_short)
# max_fft_Rot_Y_short=max(fft_Rot_Y_short)

x_short=xf
# plt.plot(fft_Rot_Y_short,fft_Rot_Y_short)

In [3]:
#Frequency prediction
#https://www.cpuheater.com/deep-learning/introduction-to-recurrent-neural-networks-in-pytorch/
#success with Rot_X

import torch
from torch.autograd import Variable
import numpy as np
import pylab as pl
import torch.nn.init as init

dtype = torch.FloatTensor
input_size, hidden_size, output_size = 7, 6, 1
epochs = 1000
# seq_length = 20
lr = 0.000001 #0.00005

# data_time_steps = np.linspace(2, 10, seq_length + 1)
# data = np.sin(data_time_steps)

x_=np.divide(fft_Rot_X_short,max(fft_Rot_X_short))
max_fft_Rot_X_short=max(fft_Rot_X_short)

data_Rot_X=x_.copy(order='C')
seq_length=len(data_Rot_X)

data_Rot_X.resize((seq_length + 1, 1))

x = Variable(torch.Tensor(data_Rot_X[:-1]).type(dtype), requires_grad=False)
y = Variable(torch.Tensor(data_Rot_X[1:]).type(dtype), requires_grad=False)

w1_Rot_X = torch.FloatTensor(input_size, hidden_size).type(dtype)
# init.normal(w1_Rot_X, 0, 0.3)
w1_Rot_X=torch.tensor([[ 0.4686, -0.2785,  0.7082, -0.0189,  0.2265,  0.8570],
        [-0.5439, -0.4637, -0.0066, -0.4237, -0.6787,  0.0386],
        [ 0.5543,  0.2468,  0.4083,  0.3409,  0.3246, -0.5786],
        [ 0.4538,  0.1002,  0.8488, -0.4980,  0.1340, -0.8546],
        [ 0.3559, -0.2763,  0.1110,  1.1380, -0.1804,  0.7301],
        [ 0.0165,  0.6896, -0.2735, -0.7036, -0.4704,  0.2348],
        [-0.0529, -0.2090,  0.6033, -0.4442,  0.5282, -0.0718]],
       requires_grad=True)
w1_Rot_X =  Variable(w1_Rot_X, requires_grad=True)

w2_Rot_X = torch.FloatTensor(hidden_size, output_size).type(dtype)
# init.normal(w2_Rot_X, 0, 0.4)

w2_Rot_X=torch.tensor([[ 0.4116],
        [ 0.1517],
        [ 0.9228],
        [ 1.0486],
        [ 0.2479],
        [-0.9111]], requires_grad=True)
w2_Rot_X = Variable(w2_Rot_X, requires_grad=True)

def forward(input, context_state, w1_Rot_X, w2_Rot_X):
  xh = torch.cat((input, context_state), 1)
  context_state = torch.tanh(xh.mm(w1_Rot_X))
  out = context_state.mm(w2_Rot_X)
  return  (out, context_state)

for i in range(epochs):
  total_loss = 0
  context_state = Variable(torch.zeros((1, hidden_size)).type(dtype), requires_grad=True)

  for j in range(x.size(0)):
    input  = x[j:(j+1)]
    target = y[j:(j+1)]
    (pred, context_state) = forward(input, context_state, w1_Rot_X, w2_Rot_X)
    loss = (pred - target).pow(2).sum()/2
    total_loss += loss
    loss.backward()
    w1_Rot_X.data -= lr * w1_Rot_X.grad.data
    w2_Rot_X.data -= lr * w2_Rot_X.grad.data
    w1_Rot_X.grad.data.zero_()
    w2_Rot_X.grad.data.zero_()
    context_state = Variable(context_state.data)
  if i % 100 == 0:
     print("Epoch: {} loss {}".format(i, total_loss.data))

context_state = Variable(torch.zeros((1, hidden_size)).type(dtype), requires_grad=False)
predictions_Rot_X_short = []

print("w1: ",w1_Rot_X)
print("w2: ",w2_Rot_X)

for i in range(x.size(0)):
  input = x[i:i+1]
  (pred, context_state) = forward(input, context_state, w1_Rot_X, w2_Rot_X)
  context_state = context_state
  predictions_Rot_X_short.append(pred.data.numpy().ravel()[0])

# plt.plot(y)
# plt.plot(predictions_Rot_X_short)

Epoch: 0 loss 5.866237640380859
Epoch: 100 loss 5.852173805236816
Epoch: 200 loss 5.8387579917907715
Epoch: 300 loss 5.825872898101807
Epoch: 400 loss 5.813634872436523
Epoch: 500 loss 5.802046298980713
Epoch: 600 loss 5.791093349456787
Epoch: 700 loss 5.780697345733643
Epoch: 800 loss 5.77076530456543
Epoch: 900 loss 5.761338710784912
w1:  tensor([[ 0.4680, -0.2787,  0.7074, -0.0204,  0.2259,  0.8570],
        [-0.5436, -0.4636, -0.0060, -0.4232, -0.6786,  0.0383],
        [ 0.5548,  0.2469,  0.4091,  0.3421,  0.3249, -0.5792],
        [ 0.4536,  0.1001,  0.8486, -0.4985,  0.1340, -0.8540],
        [ 0.3555, -0.2765,  0.1088,  1.1368, -0.1808,  0.7309],
        [ 0.0170,  0.6897, -0.2726, -0.7028, -0.4701,  0.2341],
        [-0.0544, -0.2094,  0.6015, -0.4474,  0.5277, -0.0689]],
       requires_grad=True)
w2:  tensor([[ 0.4105],
        [ 0.1549],
        [ 0.9199],
        [ 1.0481],
        [ 0.2456],
        [-0.9126]], requires_grad=True)


In [4]:
#generate predicted Rot_X wave
Rot_X_predicted=ifft(predictions_Rot_X_short[0:-1] + [0]*19580+[0]*prediction_time*20)
temp_fft_Rot_X_train_reverse=np.divide( Rot_X_predicted,max(Rot_X_predicted))
Rot_X_predicted=np.multiply(temp_fft_Rot_X_train_reverse, max_Rot_X_train)
print((Rot_X_predicted))
# print(len(Rot_Y_predicted))
# plt.plot((Rot_X_predicted),c='r')
# plt.plot(Rot_Y_train)
# plt.plot(np.multiply(temp_y_fft_Rot_Y_train_reverse,max_y_fft_Rot_Y_short))

[0.25580844-0.11391808j 0.26117398-0.10259012j 0.26712823-0.09135543j ...
 0.24261765-0.14986876j 0.24661173-0.13742619j 0.25097849-0.12548455j]


In [5]:
#Rot_X error
actual_data_Rot_X=Rot_X[0:len(Rot_X_predicted)][-20*prediction_time:]
predicted_data_Rot_X=Rot_X_predicted[-20*prediction_time:]
#error for x
ss_res_train=[]
for k in range(len(predicted_data_Rot_X)):
    ss_res_train.append((actual_data_Rot_X[k]- predicted_data_Rot_X[k])**2)
ss_tot_train=[]
for k in range(len(predicted_data_Rot_X)):
    ss_tot_train.append((actual_data_Rot_X[k]-np.mean(actual_data_Rot_X))**2)
r_squared_train_Rot_X = np.abs(1 - (np.sum( ss_res_train)/np.sum(ss_tot_train)))

# plt.plot(predicted_data_Rot_X,actual_data_Rot_X)
print("r_squared_train_Rot_X: ",np.abs(np.sum(ss_res_train))/len(predicted_data_Rot_X))

r_squared_train_Rot_X:  10.34676767566113


In [6]:
#https://www.cpuheater.com/deep-learning/introduction-to-recurrent-neural-networks-in-pytorch/
#success with Rot_X vs y


dtype = torch.FloatTensor
input_size, hidden_size, output_size = 5, 4, 1
epochs = 100
# seq_length = 20
lr = 0.00006 #0.00005

# data_time_steps = np.linspace(2, 10, seq_length + 1)
# data = np.sin(data_time_steps)

Rot_X_=np.divide(Rot_X_train,max(Rot_X_train))
# print(y_fft_short)
data_Rot_X=Rot_X_.copy(order='C')
seq_length=len(data_Rot_X)
data_Rot_X.resize((seq_length + 1, 1))
X_train_max=max(X_train)

Y_=np.divide(Y_train,max(Y_train))
# print(y_fft_short)
data_Y=Y_.copy(order='C')
seq_length=len(data_Y)
data_Y.resize((seq_length + 1, 1))

# print(np.shape(data),data)

y = Variable(torch.Tensor(data_Y[1:]).type(dtype), requires_grad=False)
rot_x = Variable(torch.Tensor(data_Rot_X[1:]).type(dtype), requires_grad=False)

# print(np.shape(x),x)

# print(np.shape(y),y)

w1_y = torch.FloatTensor(input_size, hidden_size).type(dtype)
init.normal(w1_y, 0, 0.3)
# w1_y=torch.tensor([[ 0.6040, -0.7184, -0.4734,  0.3292],
#         [ 0.4197, -0.1070,  0.0947, -0.7358],
#         [ 0.2507,  0.0215,  0.4250,  0.0370],
#         [-0.2239,  0.4435, -0.1628,  0.5767],
#         [ 0.3655,  0.0549, -0.1239,  0.2886]], requires_grad=True)
w1_y =  Variable(w1_y, requires_grad=True)
print(np.shape(w1_y))
w2_y = torch.FloatTensor(hidden_size, output_size).type(dtype)
init.normal(w2_y, 0, 0.4)
# w2_y=torch.tensor([[ 0.9871],
#         [-0.6229],
#         [-0.4600],
#         [ 0.2643]], requires_grad=True)

w2_y = Variable(w2_y, requires_grad=True)
print(np.shape(w2_y))
def forward(input, context_state, w1_y, w2_y):
  xh = torch.cat((input, context_state), 1)
  context_state = torch.tanh(xh.mm(w1_y))
  out = context_state.mm(w2_y)
  return  (out, context_state)
for i in range(epochs):
  total_loss = 0
  context_state = Variable(torch.zeros((1, hidden_size)).type(dtype), requires_grad=True)
  for j in range(y.size(0)):
    input = rot_x[j:(j+1)]
    target = y[j:(j+1)]
    (pred, context_state) = forward(input, context_state, w1_y, w2_y)
    loss = (pred - target).pow(2).sum()/2
    total_loss += loss
    loss.backward()
    w1_y.data -= lr * w1_y.grad.data
    w2_y.data -= lr * w2_y.grad.data
    w1_y.grad.data.zero_()
    w2_y.grad.data.zero_()
    context_state = Variable(context_state.data)
  if i % 20 == 0:
     print("Epoch: {} loss {}".format(i, total_loss.data))
print("w1: ",w1_y)
print("w2: ",w2_y)

torch.Size([5, 4])
torch.Size([4, 1])


<ipython-input-6-b0aed8f18188>:37: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(w1_y, 0, 0.3)
<ipython-input-6-b0aed8f18188>:46: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(w2_y, 0, 0.4)


Epoch: 0 loss 2607.40087890625
Epoch: 20 loss 2280.24267578125
Epoch: 40 loss 2271.918701171875
Epoch: 60 loss 2271.382568359375
Epoch: 80 loss 2271.184326171875
w1:  tensor([[ 0.6012,  0.6166,  0.2862,  0.1950],
        [ 0.0923,  0.0244,  0.1502,  0.5072],
        [ 0.0235,  0.3742,  0.1996, -0.6959],
        [-0.1187,  0.5691,  0.3316, -0.1720],
        [ 0.1494,  0.0198, -0.1890,  0.6726]], requires_grad=True)
w2:  tensor([[ 0.6033],
        [-0.0581],
        [ 0.4840],
        [-0.0777]], requires_grad=True)


In [7]:
#prediction Y

context_state = Variable(torch.zeros((1, hidden_size)).type(dtype), requires_grad=False)
predictions = []
Y_predict=[]
data_=Rot_X_predicted.copy(order='C')
seq_length=len(data_)
data_.resize((seq_length + 1, 1))
temp_ = Variable(torch.Tensor(data_[:-1]).type(dtype), requires_grad=False)

for i in range(temp_.size(0)):
  input = temp_[i:i+1]
  (pred, context_state) = forward(input, context_state, w1_y, w2_y)
  context_state = context_state
  predictions.append(pred.data.numpy().ravel()[0])

Y_predict=np.divide(predictions,max(predictions))
Y_predict=np.multiply(Y_predict,max_Y_train)
# plt.plot(Y)
# plt.plot(Y_predict)

In [8]:
#MSE error:

#X,Y,Z error

actual_data_y=Y[0:len(Y_predict)][-20*prediction_time:]

predicted_data_y=Y_predict[-20*prediction_time:]

#error for y
ss_res_train=[]
for k in range(len(predicted_data_y)):
    ss_res_train.append((actual_data_y[k]- predicted_data_y[k])**2)
ss_tot_train=[]
for k in range(len(predicted_data_y)):
    ss_tot_train.append((actual_data_y[k]-np.mean(actual_data_y))**2)
r_squared_train_y = 1 - (np.sum( ss_res_train)/np.sum(ss_tot_train))

print("r_squared_train_y: ",np.sum( ss_res_train),np.sum(ss_tot_train))

r_squared_train_y:  25576.16769397355 1917.4946513360949


In [9]:
#Frequency prediction
#https://www.cpuheater.com/deep-learning/introduction-to-recurrent-neural-networks-in-pytorch/
#success with Rot_Y

import torch
from torch.autograd import Variable
import numpy as np
import pylab as pl
import torch.nn.init as init

dtype = torch.FloatTensor
input_size, hidden_size, output_size = 7, 6, 1
epochs = 500
# seq_length = 20
lr = 0.00005 #0.00005

# data_time_steps = np.linspace(2, 10, seq_length + 1)
# data = np.sin(data_time_steps)

y_=np.divide(fft_Rot_Y_short,max(fft_Rot_Y_short))
max_y_fft_Rot_Y_short=max(fft_Rot_Y_short)

data=y_.copy(order='C')
seq_length=len(data)

data.resize((seq_length + 1, 1))

x = Variable(torch.Tensor(data[:-1]).type(dtype), requires_grad=False)
y = Variable(torch.Tensor(data[1:]).type(dtype), requires_grad=False)

w1_Rot_Y = torch.FloatTensor(input_size, hidden_size).type(dtype)
# init.normal(w1, 0, 0.3)
w1_Rot_Y=torch.tensor([[-0.8976, -0.0443, -0.3203,  0.1730, -0.0658, -0.2087],
        [ 0.0131, -0.1364, -0.0186,  0.2386,  0.0117, -0.3229],
        [ 0.1395, -0.0851,  0.2515,  0.0590,  0.4586,  0.1279],
        [ 0.5605, -0.1500, -0.2014, -0.1581,  0.1909, -0.0185],
        [-0.1277, -0.3580, -0.0395, -0.2905, -0.0605,  0.0347],
        [ 0.4607, -0.0569, -0.1329, -0.1055,  0.0483,  0.0342],
        [-0.1916, -0.0462, -0.3716,  0.1053, -0.1005, -0.7972]],requires_grad=True)
w1_Rot_Y =  Variable(w1_Rot_Y, requires_grad=True)
print("w1: ",w1_Rot_Y)

w2_Rot_Y = torch.FloatTensor(hidden_size, output_size).type(dtype)
# init.normal(w2, 0, 0.4)
w2_Rot_Y=torch.tensor([[ 0.3457],
        [-0.7237],
        [ 0.1072],
        [ 0.8113],
        [-0.6045],
        [-0.2506]], requires_grad=True)
w2_Rot_Y = Variable(w2_Rot_Y, requires_grad=True)
print("w2: ",w2_Rot_Y)

def forward(input, context_state, w1_Rot_Y, w2_Rot_Y):
  xh = torch.cat((input, context_state), 1)
  context_state = torch.tanh(xh.mm(w1_Rot_Y))
  out = context_state.mm(w2_Rot_Y)
  return  (out, context_state)

for i in range(epochs):
  total_loss = 0
  context_state = Variable(torch.zeros((1, hidden_size)).type(dtype), requires_grad=True)

  for j in range(x.size(0)):
    input = x[j:(j+1)]
    target = y[j:(j+1)]
    (pred, context_state) = forward(input, context_state, w1_Rot_Y, w2_Rot_Y)
    loss = (pred - target).pow(2).sum()/2
    total_loss += loss
    loss.backward()
    w1_Rot_Y.data -= lr * w1_Rot_Y.grad.data
    w2_Rot_Y.data -= lr * w2_Rot_Y.grad.data
    w1_Rot_Y.grad.data.zero_()
    w2_Rot_Y.grad.data.zero_()
    context_state = Variable(context_state.data)
  if i % 100 == 0:
     print("Epoch: {} loss {}".format(i, total_loss.data))

context_state = Variable(torch.zeros((1, hidden_size)).type(dtype), requires_grad=False)
predictions_Rot_Y_short = []

for i in range(x.size(0)):
  input = x[i:i+1]
  (pred, context_state) = forward(input, context_state, w1_Rot_Y, w2_Rot_Y)
  context_state = context_state
  predictions_Rot_Y_short.append(pred.data.numpy().ravel()[0])
    
# plt.plot(y_)
# plt.plot(predictions_Rot_Y_short)

w1:  tensor([[-0.8976, -0.0443, -0.3203,  0.1730, -0.0658, -0.2087],
        [ 0.0131, -0.1364, -0.0186,  0.2386,  0.0117, -0.3229],
        [ 0.1395, -0.0851,  0.2515,  0.0590,  0.4586,  0.1279],
        [ 0.5605, -0.1500, -0.2014, -0.1581,  0.1909, -0.0185],
        [-0.1277, -0.3580, -0.0395, -0.2905, -0.0605,  0.0347],
        [ 0.4607, -0.0569, -0.1329, -0.1055,  0.0483,  0.0342],
        [-0.1916, -0.0462, -0.3716,  0.1053, -0.1005, -0.7972]],
       requires_grad=True)
w2:  tensor([[ 0.3457],
        [-0.7237],
        [ 0.1072],
        [ 0.8113],
        [-0.6045],
        [-0.2506]], requires_grad=True)
Epoch: 0 loss 4.5810699462890625
Epoch: 100 loss 4.484375476837158
Epoch: 200 loss 4.4062323570251465
Epoch: 300 loss 4.342918395996094
Epoch: 400 loss 4.291516304016113


In [10]:
#generate predicted wave
Rot_Y_predicted=ifft([0]*100+predictions_Rot_Y_short[100:-1] + [0]*19580+[0]*prediction_time*20)
temp_fft_Rot_Y_train_reverse=np.divide( Rot_Y_predicted,max(Rot_Y_predicted))
Rot_Y_predicted=np.multiply(temp_fft_Rot_Y_train_reverse, max_Rot_Y_train)
# print((Rot_Y_predicted))
# print(len(Rot_Y_predicted))
# plt.plot(Rot_Y_predicted,c='r')
# plt.plot(Rot_Y_train)
# plt.plot(np.multiply(temp_y_fft_Rot_Y_train_reverse,max_y_fft_Rot_Y_short))

In [11]:
#https://www.cpuheater.com/deep-learning/introduction-to-recurrent-neural-networks-in-pytorch/
#success with Rot_Y vs z


dtype = torch.FloatTensor
input_size, hidden_size, output_size = 5, 4, 1
epochs = 100
# seq_length = 20
lr = 0.00006 #0.00005

# data_time_steps = np.linspace(2, 10, seq_length + 1)
# data = np.sin(data_time_steps)

Rot_Y_=np.divide(Rot_Y_train,max(Rot_Y_train))
# print(y_fft_short)
data_Rot_Y=Rot_Y_.copy(order='C')
seq_length=len(data_Rot_Y)
data_Rot_Y.resize((seq_length + 1, 1))
Y_train_max=max(Y_train)

Z_=np.divide(Z_train,max(Z_train))
# print(y_fft_short)
data_Z=Z_.copy(order='C')
seq_length=len(data_Z)
data_Z.resize((seq_length + 1, 1))

# print(np.shape(data),data)

z = Variable(torch.Tensor(data_Z[1:]).type(dtype), requires_grad=False)
y = Variable(torch.Tensor(data_Rot_Y[1:]).type(dtype), requires_grad=False)

# print(np.shape(x),x)

# print(np.shape(y),y)

w1_z = torch.FloatTensor(input_size, hidden_size).type(dtype)
# init.normal(w1, 0, 0.3)
w1_z=torch.tensor([[ 0.6040, -0.7184, -0.4734,  0.3292],
        [ 0.4197, -0.1070,  0.0947, -0.7358],
        [ 0.2507,  0.0215,  0.4250,  0.0370],
        [-0.2239,  0.4435, -0.1628,  0.5767],
        [ 0.3655,  0.0549, -0.1239,  0.2886]], requires_grad=True)
w1_z =  Variable(w1_z, requires_grad=True)
print(np.shape(w1_z))
w2_z = torch.FloatTensor(hidden_size, output_size).type(dtype)
# init.normal(w2, 0, 0.4)
w2_z=torch.tensor([[ 0.9871],
        [-0.6229],
        [-0.4600],
        [ 0.2643]], requires_grad=True)

w2_z = Variable(w2_z, requires_grad=True)
print(np.shape(w2_z))
def forward(input, context_state, w1, w2):
  xh = torch.cat((input, context_state), 1)
  context_state = torch.sigmoid(xh.mm(w1))
  out = context_state.mm(w2)
  return  (out, context_state)
for i in range(epochs):
  total_loss = 0
  context_state = Variable(torch.zeros((1, hidden_size)).type(dtype), requires_grad=True)
  for j in range(z.size(0)):
    input = y[j:(j+1)]
    target = z[j:(j+1)]
    (pred, context_state) = forward(input, context_state, w1_z, w2_z)
    loss = (pred - target).pow(2).sum()/2
    total_loss += loss
    loss.backward()
    w1_z.data -= lr * w1_z.grad.data
    w2_z.data -= lr * w2_z.grad.data
    w1_z.grad.data.zero_()
    w2_z.grad.data.zero_()
    context_state = Variable(context_state.data)
  if i % 100 == 0:
     print("Epoch: {} loss {}".format(i, total_loss.data))
print("w1: ",w1_z)
print("w2: ",w2_z)

torch.Size([5, 4])
torch.Size([4, 1])
Epoch: 0 loss 210794.734375
w1:  tensor([[-0.1625, -2.1598, -1.9456, -1.1110],
        [ 1.3065, -0.1157,  0.1000, -0.3820],
        [ 1.0104,  0.2343,  0.6588,  0.5346],
        [ 0.5178,  0.6113,  0.0239,  1.0314],
        [ 1.0372, -0.0249, -0.1954,  0.5033]], requires_grad=True)
w2:  tensor([[3.7694],
        [2.4965],
        [2.5038],
        [2.7719]], requires_grad=True)


In [12]:
#prediction z

context_state = Variable(torch.zeros((1, hidden_size)).type(dtype), requires_grad=False)
predictions = []

data_=Rot_Y_predicted.copy(order='C')
seq_length=len(data_)
data_.resize((seq_length + 1, 1))
temp_ = Variable(torch.Tensor(data_[:-1]).type(dtype), requires_grad=False)

for i in range(temp_.size(0)):
  input = temp_[i:i+1]
  (pred, context_state) = forward(input, context_state, w1_z, w2_z)
  context_state = context_state
  predictions.append(pred.data.numpy().ravel()[0])

z_predict=np.divide(predictions,max(predictions))
z_predict=np.multiply(z_predict,max_Z_train)
# plt.plot(y_)
# plt.plot(z_predict)

In [13]:
# predict X
# Insert Modeling Building or Plotting code here
# Note, you may implement these however you see fit
# Ex: using an existing library, solving the Normal Eqns
#     implementing your own SGD solver for them. Your Choice.


#Load data
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_openml
from datetime import datetime, timedelta
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

X_=time_train.to_numpy()
y_=X_train
X_ = X_.reshape(-1,1)

X_train_=X_
y_train_=y_
# X_test=time[:len(X_train)+prediction_time*20]
X_test_=X_
y_test_=y_

print(len(X_train_))

import pandas
from sklearn.pipeline import Pipeline
from scipy.optimize import curve_fit
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
def test_func1(x_, dist, b):
    return dist + b* x_

ratio=0.97752809
plt.figure(figsize=(10,10))
n=1
r_squared_array=np.zeros(shape=(6,4))
r_squared_array_train=np.zeros(shape=(6,4))

# plt.subplot(6,4,n)
y_=[]
x=[]
# X_train, X_test, y_train, y_test = train_test_split(
#  X, y, test_size=ratio, random_state=0)   #0.101123595505618,0.97752808988764



for i in X_train_:
   x.append(i[0]) 

# plt.plot(x,y_train_)
n=n+1
fy_test=[]
fy_train=[]
params, params_covariance = curve_fit(test_func1, x, y_train_)
for i in X_test_:
    fy_test.append(test_func1(i,*params))
for i in X_train_:
    fy_train.append(test_func1(i,*params))

text= 'Linear'
ss_res=[]
for k in range(len(fy_test)):
    ss_res.append((y_test_[k]- fy_test[k])**2)
ss_tot=[]
for k in range(len(fy_test)):
    ss_tot.append((y_test_[k]-np.mean(y_test_))**2)
r_squared = 1 - (np.sum( ss_res)/np.sum(ss_tot))

ss_res_train=[]
for k in range(len(fy_train)):
    ss_res_train.append((y_train_[k]- fy_train[k])**2)
ss_tot_train=[]
for k in range(len(fy_train)):
    ss_tot_train.append((y_train_[k]-np.mean(y_train_))**2)
r_squared_train = 1 - (np.sum( ss_res_train)/np.sum(ss_tot_train))
r_squared_array=r_squared
r_squared_array_train=r_squared_train

temp=[i[0] for i in ss_res]
X_test_=[i[0] for i in X_test_]

print(len(X_)+prediction_time*20)

x_test_= time.to_numpy()[:len(time_train)+prediction_time*20]
print(len(x_test_))
Y_test_= y_

print(len(X_train_),len(fy_train))

X_predict=params[0]+params[1]*x_test_

# plt.scatter(x_test_, X_predict,s=10,c='Black')
# plt.scatter(X_train_, fy_train,s=10,c='Red')

# plt.title(" Type: "+text+"\nTrain score = {:.3f}\n N of test size = {:.0f}".format( r_squared,2225-ratio*2225))
# print(params)
velocity=params[1]*0.3048
print("velocity (m/s): ",velocity)
# plt.scatter([1,2,3,4], r_squared_array,s=10,c='Black')
# plt.title(" Type: "+text+"\nTrain score = {:.3f}\n N of test size = {:.0f}".format( r_squared,2225-ratio[j]*2225))
# plt.show()

19980
20080
20080
19980 19980
velocity (m/s):  15.398073542655387


<Figure size 720x720 with 0 Axes>

In [14]:
#Frequency prediction
#https://www.cpuheater.com/deep-learning/introduction-to-recurrent-neural-networks-in-pytorch/
#success with Rot_Z

import torch
from torch.autograd import Variable
import numpy as np
import pylab as pl
import torch.nn.init as init

dtype = torch.FloatTensor
input_size, hidden_size, output_size = 6, 5, 1
epochs = 500
# seq_length = 20
lr = 0.00005 #0.00005

# data_time_steps = np.linspace(2, 10, seq_length + 1)
# data = np.sin(data_time_steps)

y_=np.divide(fft_Rot_Z_short,max(fft_Rot_Z_short))
max_fft_Rot_Z_short=max(fft_Rot_Z_short)

data=y_.copy(order='C')
seq_length=len(data)

data.resize((seq_length + 1, 1))

x = Variable(torch.Tensor(data[:-1]).type(dtype), requires_grad=False)
y = Variable(torch.Tensor(data[1:]).type(dtype), requires_grad=False)

w1_Rot_Z = torch.FloatTensor(input_size, hidden_size).type(dtype)
init.normal(w1_Rot_Z, 0, 0.3)
# w1_Rot_Z=torch.tensor([[-0.8976, -0.0443, -0.3203,  0.1730, -0.0658, -0.2087],
#         [ 0.0131, -0.1364, -0.0186,  0.2386,  0.0117, -0.3229],
#         [ 0.1395, -0.0851,  0.2515,  0.0590,  0.4586,  0.1279],
#         [ 0.5605, -0.1500, -0.2014, -0.1581,  0.1909, -0.0185],
#         [-0.1277, -0.3580, -0.0395, -0.2905, -0.0605,  0.0347],
#         [ 0.4607, -0.0569, -0.1329, -0.1055,  0.0483,  0.0342],
#         [-0.1916, -0.0462, -0.3716,  0.1053, -0.1005, -0.7972]],
#        requires_grad=True)
w1_Rot_Z =  Variable(w1_Rot_Z, requires_grad=True)
print("w1: ",w1_Rot_Z)

w2_Rot_Z = torch.FloatTensor(hidden_size, output_size).type(dtype)
init.normal(w2_Rot_Z, 0, 0.4)
# w2_Rot_Z=torch.tensor([[ 0.3457],
#         [-0.7237],
#         [ 0.1072],
#         [ 0.8113],
#         [-0.6045],
#         [-0.2506]], requires_grad=True)
w2_Rot_Z = Variable(w2_Rot_Z, requires_grad=True)
print("w2: ",w2_Rot_Z)

def forward(input, context_state, w1_Rot_Z, w2_Rot_Z):
  xh = torch.cat((input, context_state), 1)
  context_state = torch.tanh(xh.mm(w1_Rot_Z))
  out = context_state.mm(w2_Rot_Z)
  return  (out, context_state)

for i in range(epochs):
  total_loss = 0
  context_state = Variable(torch.zeros((1, hidden_size)).type(dtype), requires_grad=True)

  for j in range(x.size(0)):
    input = x[j:(j+1)]
    target = y[j:(j+1)]
    (pred, context_state) = forward(input, context_state, w1_Rot_Z, w2_Rot_Z)
    loss = (pred - target).pow(2).sum()/2
    total_loss += loss
    loss.backward()
    w1_Rot_Z.data -= lr * w1_Rot_Z.grad.data
    w2_Rot_Z.data -= lr * w2_Rot_Z.grad.data
    w1_Rot_Z.grad.data.zero_()
    w2_Rot_Z.grad.data.zero_()
    context_state = Variable(context_state.data)
  if i % 100 == 0:
     print("Epoch: {} loss {}".format(i, total_loss.data))

context_state = Variable(torch.zeros((1, hidden_size)).type(dtype), requires_grad=False)
predictions_Rot_Z_short = []

for i in range(x.size(0)):
  input = x[i:i+1]
  (pred, context_state) = forward(input, context_state, w1_Rot_Z, w2_Rot_Z)
  context_state = context_state
  predictions_Rot_Z_short.append(pred.data.numpy().ravel()[0])
    
# plt.plot(y_)
# plt.plot(predictions_Rot_Z_short)

w1:  tensor([[-0.0772,  0.6960, -0.4124,  0.0416, -0.2429],
        [-0.0684, -0.3990,  0.5290,  0.0226,  0.2052],
        [ 0.2327, -0.5188,  0.0701, -0.3986, -0.2465],
        [-0.2093,  0.4494,  0.3747,  0.1062,  0.2342],
        [-0.2192, -0.3744,  0.0869, -0.1836, -0.4926],
        [ 0.1669, -0.5710,  0.1060, -0.2503, -0.0245]], requires_grad=True)
w2:  tensor([[ 0.2551],
        [-0.4843],
        [-0.0811],
        [ 0.0319],
        [-0.1918]], requires_grad=True)


<ipython-input-14-ebcd758ffe3e>:32: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(w1_Rot_Z, 0, 0.3)
<ipython-input-14-ebcd758ffe3e>:45: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(w2_Rot_Z, 0, 0.4)


Epoch: 0 loss 14.192745208740234
Epoch: 100 loss 3.489047050476074
Epoch: 200 loss 2.001464605331421
Epoch: 300 loss 1.489052176475525
Epoch: 400 loss 1.226842999458313


In [15]:
#generate predicted Rot_Z wave
Rot_Z_predicted=ifft(predictions_Rot_Z_short[0:-1] + [0]*19580+[0]*prediction_time*20)
temp_fft_Rot_Z_train_reverse=np.divide( Rot_Z_predicted,max(Rot_Z_predicted))
Rot_Z_predicted=np.multiply(temp_fft_Rot_Z_train_reverse, max_Rot_Z_train)
print((max_Rot_Z_train))
# print(len(Rot_Y_predicted))
# plt.plot((Rot_Z_predicted),c='r')
# plt.plot(Rot_Z_train)
# plt.plot(np.multiply(temp_y_fft_Rot_Y_train_reverse,max_y_fft_Rot_Y_short))

3.3753326


In [16]:
#Rot_X,Y,Z error

actual_data_Rot_Z=Rot_Z[0:len(Rot_Z_predicted)][-20*prediction_time:]

predicted_data_Rot_Z=Rot_Z_predicted[-20*prediction_time:]

#error for z
ss_res_train_Rot_Z=[]
for k in range(len(predicted_data_Rot_Z)):
    ss_res_train_Rot_Z.append(np.abs((actual_data_Rot_Z[k]- predicted_data_Rot_Z[k])**2))


print("MSE_Rot_Z: ",np.power(sum(ss_res_train_Rot_Z),0.5)/len(ss_res_train_Rot_Z))

MSE_Rot_Z:  0.06003781993471812


# OUTPUT

In [17]:
#MSE error:

#X,Y,Z error

actual_data_x=X[0:len(X_predict)][-20*prediction_time:]
actual_data_y=Y[0:len(Y_predict)][-20*prediction_time:]
actual_data_z=Z[0:len(z_predict)][-20*prediction_time:]

predicted_data_x=X_predict[-20*prediction_time:]
predicted_data_y=Y_predict[-20*prediction_time:]
predicted_data_z=z_predict[-20*prediction_time:]

# plt.plot(predicted_data_x,actual_data_x)

#error for x
ss_res_train_x=[]
for k in range(len(predicted_data_x)):
    ss_res_train_x.append((actual_data_x[k]- predicted_data_x[k])**2)

#error for y
ss_res_train_y=[]
for k in range(len(predicted_data_y)):
    ss_res_train_y.append((actual_data_y[k]- predicted_data_y[k])**2)

#error for z
ss_res_train_z=[]
for k in range(len(predicted_data_z)):
    ss_res_train_z.append((actual_data_z[k]- predicted_data_z[k])**2)

print("MSE_x: ",np.power(sum(ss_res_train_x),0.5)/len(predicted_data_x))
print("MSE_y: ",np.power(sum(ss_res_train_y),0.5)/len(predicted_data_y))
print("MSE_z: ",np.power(sum(ss_res_train_z),0.5)/len(predicted_data_z))


MSE_x:  1.8514792469534336
MSE_y:  1.5992550670225671
MSE_z:  1.6088021417915113


In [18]:
#Rot_X,Y,Z error

actual_data_Rot_X=Rot_X[0:len(Rot_X_predicted)][-20*prediction_time:]
actual_data_Rot_Y=Rot_Y[0:len(Rot_Y_predicted)][-20*prediction_time:]
actual_data_Rot_Z=Rot_Z[0:len(Rot_Z_predicted)][-20*prediction_time:]

predicted_data_Rot_X=Rot_X_predicted[-20*prediction_time:]
predicted_data_Rot_Y=Rot_Y_predicted[-20*prediction_time:]
predicted_data_Rot_Z=Rot_Z_predicted[-20*prediction_time:]


#error for x
ss_res_train_Rot_X=[]
for k in range(len(predicted_data_Rot_X)):
    ss_res_train_Rot_X.append(np.abs((actual_data_Rot_X[k]- predicted_data_Rot_X[k])**2))

#error for y
ss_res_train_Rot_Y=[]
for k in range(len(predicted_data_Rot_Y)):
    ss_res_train_Rot_Y.append(np.abs((actual_data_Rot_Y[k]- predicted_data_Rot_Y[k])**2))

#error for z
ss_res_train_Rot_Z=[]
for k in range(len(predicted_data_Rot_Z)):
    ss_res_train_Rot_Z.append(np.abs((actual_data_Rot_Z[k]- predicted_data_Rot_Z[k])**2))



print("MSE_Rot_X: ",np.power(sum(ss_res_train_Rot_X),0.5)/len(ss_res_train_Rot_X))
print("MSE_Rot_Y: ",np.power(sum(ss_res_train_Rot_Y),0.5)/len(ss_res_train_Rot_Y))
print("MSE_Rot_Z: ",np.power(sum(ss_res_train_Rot_Z),0.5)/len(ss_res_train_Rot_Z))

print("Time to move 1 meter: ",1/velocity)

MSE_Rot_X:  0.3260607043373058
MSE_Rot_Y:  0.22075573439354623
MSE_Rot_Z:  0.06003781993471812
Time to move 1 meter:  0.06494318897944104
